In [1]:
# ========== KIỂM TRA VÀ CÀI ĐẶT THƯ VIỆN ==========
# Mục đích: Tự động phát hiện môi trường (Kaggle hoặc Local) và cài đặt thư viện cần thiết

import os
import subprocess
import csv
import json
import warnings
import time

# Phát hiện môi trường: Kiểm tra thư mục đặc trưng của Kaggle
IS_KAGGLE = os.path.exists('/kaggle/input')
print(f"Môi trường: {'Kaggle' if IS_KAGGLE else 'Local'}")

if IS_KAGGLE:
    print("Đang cài đặt các thư viện cần thiết...")
    
    # Cài đặt seqeval: Thư viện tính metrics cho NER (Named Entity Recognition)
    try:
        import seqeval
        pass
    except ImportError:
        subprocess.check_call(['pip', 'install', '-q', 'seqeval'])
        print("Đã cài seqeval")
    
    # Kiểm tra transformers và torch đã được cài sẵn trên Kaggle
    import transformers
    import torch
    pass
else:
    print("Môi trường Local: Cần cài đặt: pip install transformers seqeval torch")

Môi trường: Kaggle
Đang cài đặt các thư viện cần thiết...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
Đã cài seqeval


## 1. Import thư viện

In [2]:
# ========== IMPORT THƯ VIỆN ==========

# Xử lý dữ liệu
import pandas as pd  # Đọc và xử lý file CSV
import numpy as np   # Tính toán số học

# Deep Learning với PyTorch
import torch
import torch.nn as nn  # Xây dựng mạng neural
import torch.nn.functional as F  # Hàm activation và loss functions
from torch.utils.data import Dataset, DataLoader  # Tạo dataset và dataloader

# Transformers (Hugging Face) - Làm việc với PhoBERT
from transformers import (
    AutoTokenizer,           # Tokenizer tự động (tách từ)
    AutoModel,               # Load model pretrained
    AutoConfig,              # Cấu hình model
    RobertaModel,            # PhoBERT base model (cần cho Fusion)
    TrainingArguments,       # Cấu hình training
    Trainer,                 # Trainer để train model
    EarlyStoppingCallback,   # Dừng sớm khi model không cải thiện
    TrainerCallback          # Base class cho custom callbacks
)

# Scikit-learn - Chia dữ liệu và tính metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

# Seqeval - Tính metrics cho NER (Named Entity Recognition)
from seqeval.metrics import classification_report as ner_classification_report
from seqeval.metrics import f1_score as ner_f1_score
from seqeval.metrics import precision_score as ner_precision_score
from seqeval.metrics import recall_score as ner_recall_score

# Tắt cảnh báo không cần thiết
import warnings
warnings.filterwarnings('ignore')

# Kiểm tra GPU: Sử dụng GPU nếu có, không thì dùng CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Thiết bị: {device}")
print(f"PyTorch version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

2025-11-30 02:56:11.826582: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764471372.054119      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764471372.118760      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Thiết bị: cuda
PyTorch version: 2.6.0+cu124
GPU: Tesla P100-PCIE-16GB


## 2. Đọc dữ liệu

In [3]:
# Tự động chọn đường dẫn CSV dựa trên môi trường
if IS_KAGGLE:
    csv_path = "/kaggle/input/tlcn-mangxahoi/FB_TikTok_Label.csv"
else:
    csv_path = r"d:\Data_Engineer\TLCN\Crawl\fb_crawl\test_model\data_model_label\FB_TikTok_Label.csv"

print(f"CSV path: {csv_path}")

# Đọc file với csv module để xử lý dấu phẩy trong text
data_rows = []
with open(csv_path, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)  # Đọc header
    
    for i, row in enumerate(reader, start=2):  # Start từ 2 vì dòng 1 là header
        if len(row) == 5:  # Đảm bảo có đủ 5 cột
            data_rows.append(row)
        else:
            pass

# Tạo DataFrame
df = pd.DataFrame(data_rows, columns=header)

print(f"Tổng số mẫu hợp lệ: {len(df)}")
print(f"Các cột trong dataset: {df.columns.tolist()}")
print(f"Mẫu dữ liệu:")
print(df[['Description_Normalized', 'Label_NER']].head(3))

CSV path: /kaggle/input/tlcn-mangxahoi/FB_TikTok_Label.csv
Tổng số mẫu hợp lệ: 2329
Các cột trong dataset: ['\ufeffID', 'Description_Normalized', 'Label_NER', 'Label_Topic', 'Label_Intent']
Mẫu dữ liệu:
                              Description_Normalized  \
0  Chạy nhanh còn kịp ! Top ngành không nên học :...   
1       Học Quản trị Nhân_sự thì sao ? huongnghiep .   
2  Nếu tốt_nghiệp 4 chuyên_ngành này của VMU ( Đạ...   

                                           Label_NER  
0                  O O O O O O O O O O O B-MAJ O O O  
1                      O B-MAJ I-MAJ I-MAJ O O O O O  
2  O O O O O O B-ORG O B-ORG I-ORG I-ORG O O O O ...  


## 3. Tiền xử lý dữ liệu

In [4]:
def parse_multi_task_data(df):
    """
    Hàm chuyển đổi dữ liệu từ DataFrame sang format phù hợp cho Multi-task Learning
    
    Multi-task Learning: Huấn luyện model để thực hiện 3 nhiệm vụ đồng thời:
        1. NER (Named Entity Recognition) - Nhận diện thực thể
        2. Topic Classification - Phân loại chủ đề
        3. Intent Classification - Phân loại ý định
    
    Args:
        df: DataFrame chứa dữ liệu từ CSV
    
    Returns:
        texts: list các câu (mỗi câu là list các từ)
        ner_labels: list các nhãn NER (tương ứng từng từ)
        topic_labels: list các nhãn Topic (có thể multi-label: "ADMISSION|MAJOR")
        intent_labels: list các nhãn Intent (single-label)
    """
    texts = []
    ner_labels = []
    topic_labels = []
    intent_labels = []
    
    # Duyệt qua từng dòng trong DataFrame
    for idx, row in df.iterrows():
        text = row['Description_Normalized']      # Câu văn đã được chuẩn hóa
        ner_label_str = row['Label_NER']         # Nhãn NER (VD: "O O B-ORG I-ORG O")
        topic_label_str = row['Label_Topic']     # Nhãn Topic (VD: "ADMISSION|MAJOR")
        intent_label_str = row['Label_Intent']   # Nhãn Intent (VD: "ask_info")
        
        # Tách text và NER labels thành list
        words = text.split()           # ["Em", "muốn", "thi", "vào", "..."]
        ner_tags = ner_label_str.split()  # ["O", "O", "O", "O", "B-ORG", ...]
        
        # Kiểm tra tính hợp lệ: Số lượng từ phải bằng số lượng nhãn NER
        if len(words) == len(ner_tags):
            texts.append(words)
            ner_labels.append(ner_tags)
            topic_labels.append(topic_label_str)  # Giữ nguyên string (sẽ xử lý sau)
            intent_labels.append(intent_label_str)
        else:
            # Bỏ qua dòng không hợp lệ và in cảnh báo
            print(f"Cảnh báo: Dòng {idx} không khớp - số từ: {len(words)}, số nhãn: {len(ner_tags)}")
    
    return texts, ner_labels, topic_labels, intent_labels

# Parse dữ liệu
texts, ner_labels, topic_labels, intent_labels = parse_multi_task_data(df)
print(f"Số lượng câu: {len(texts)}")
print(f"Ví dụ câu 1:")
print(f"  Text: {' '.join(texts[0])}")
print(f"  NER Labels: {' '.join(ner_labels[0])}")
print(f"  Topic Label: {topic_labels[0]}")
print(f"  Intent Label: {intent_labels[0]}")

Số lượng câu: 2329
Ví dụ câu 1:
  Text: Chạy nhanh còn kịp ! Top ngành không nên học : Điều_dưỡng như_thế_nào ? .
  NER Labels: O O O O O O O O O O O B-MAJ O O O
  Topic Label: MAJOR
  Intent Label: share_info


In [5]:
# Tạo ánh xạ nhãn cho NER
unique_ner_labels = set()
for label_seq in ner_labels:
    unique_ner_labels.update(label_seq)

unique_ner_labels = sorted(list(unique_ner_labels))
ner_label2id = {label: idx for idx, label in enumerate(unique_ner_labels)}
ner_id2label = {idx: label for label, idx in ner_label2id.items()}

print(f"Số lượng nhãn NER: {len(unique_ner_labels)}")
print(f"Danh sách nhãn NER: {unique_ner_labels[:10]}...")

# Tạo ánh xạ nhãn cho Topic (xử lý multi-label)
# Tách các topic riêng lẻ từ format "ADMISSION|MAJOR"
all_topics = set()
for topic_str in topic_labels:
    topics = topic_str.split('|')
    all_topics.update(topics)

unique_topics = sorted(list(all_topics))
topic_label2id = {label: idx for idx, label in enumerate(unique_topics)}
topic_id2label = {idx: label for label, idx in topic_label2id.items()}

print(f"Số lượng topic labels: {len(unique_topics)}")
print(f"Danh sách topics: {unique_topics}")

# Tạo ánh xạ nhãn cho Intent
unique_intents = sorted(list(set(intent_labels)))
intent_label2id = {label: idx for idx, label in enumerate(unique_intents)}
intent_id2label = {idx: label for label, idx in intent_label2id.items()}

print(f"Số lượng intent labels: {len(unique_intents)}")
print(f"Danh sách intents: {unique_intents}")

Số lượng nhãn NER: 25
Danh sách nhãn NER: ['B-DATE', 'B-EX', 'B-FEE', 'B-LOC', 'B-MAJ', 'B-MISC', 'B-ORG', 'B-PRO', 'B-SAL', 'B-SCO']...
Số lượng topic labels: 10
Danh sách topics: ['CAREER', 'CERTIFICATE', 'LANGUAGE', 'MAJOR', 'OTHER', 'STUDENT_LIFE', 'STUDY', 'SUBJECT_COMBINATION', 'TUITION', 'UNIVERSITY']
Số lượng intent labels: 7
Danh sách intents: ['ask_advice', 'ask_comparison', 'ask_confirmation', 'ask_experience', 'ask_info', 'other', 'share_info']


## 4. Chia dữ liệu

In [7]:
from sklearn.model_selection import train_test_split

# ---------------------------------------------------------
# 1) Chia Train (70%) + Remaining (30%)
# ---------------------------------------------------------
train_texts, temp_texts, \
train_ner_labels, temp_ner_labels, \
train_topic_labels, temp_topic_labels, \
train_intent_labels, temp_intent_labels = train_test_split(
    texts, ner_labels, topic_labels, intent_labels,
    test_size=0.30, random_state=42
)

# ---------------------------------------------------------
# 2) Chia remaining thành Validation (15%) và Test (15%)
#    => 0.5 * 30% = 15%
# ---------------------------------------------------------
val_texts, test_texts, \
val_ner_labels, test_ner_labels, \
val_topic_labels, test_topic_labels, \
val_intent_labels, test_intent_labels = train_test_split(
    temp_texts, temp_ner_labels, temp_topic_labels, temp_intent_labels,
    test_size=0.50, random_state=42
)

# ---------------------------------------------------------
# In thống kê
# ---------------------------------------------------------
print("="*80)
print("DATA SPLIT: TRAIN / VAL / TEST")
print("="*80)

print(f"Train set:      {len(train_texts):5d} samples ({len(train_texts)/len(texts)*100:.1f}%)")
print(f"Validation set: {len(val_texts):5d} samples ({len(val_texts)/len(texts)*100:.1f}%)")
print(f"Test set:       {len(test_texts):5d} samples ({len(test_texts)/len(texts)*100:.1f}%)")
print("="*80)

print("\nLƯU Ý:")
print("  - Train: model học")
print("  - Validation: chọn best model, early stopping, tune hyperparameters")
print("  - Test: đánh giá cuối cùng, không dùng trong training/validation")
print("="*80)


DATA SPLIT: TRAIN / VAL / TEST
Train set:       1630 samples (70.0%)
Validation set:   349 samples (15.0%)
Test set:         350 samples (15.0%)

LƯU Ý:
  - Train: model học
  - Validation: chọn best model, early stopping, tune hyperparameters
  - Test: đánh giá cuối cùng, không dùng trong training/validation


## 6. Tạo Dataset

In [8]:
# Load tokenizer từ PhoBERT
phobert_path = "vinai/phobert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(phobert_path)


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [9]:
class MultiTaskDataset(Dataset):
    """
    Custom Dataset cho Multi-task Learning
    
    Chức năng:
        - Tokenize văn bản tiếng Việt bằng PhoBERT tokenizer
        - Align NER labels với subword tokens (PhoBERT tách từ thành subwords)
        - Chuyển đổi topic labels thành multi-hot vector
        - Chuyển đổi intent labels thành integer ID
        - Padding/Truncate để tất cả sequences có cùng độ dài
    """
    def __init__(self, texts, ner_labels, topic_labels, intent_labels, 
                 tokenizer, ner_label2id, topic_label2id, intent_label2id, max_length=256):
        """
        Args:
            texts: List các câu (mỗi câu là list các từ)
            ner_labels: List các nhãn NER tương ứng
            topic_labels: List các nhãn Topic (string có thể chứa | để ngăn cách)
            intent_labels: List các nhãn Intent (string)
            tokenizer: PhoBERT tokenizer
            ner_label2id: Dictionary ánh xạ NER label -> ID
            topic_label2id: Dictionary ánh xạ Topic label -> ID
            intent_label2id: Dictionary ánh xạ Intent label -> ID
            max_length: Độ dài tối đa của sequence (padding/truncate)
        """
        self.texts = texts
        self.ner_labels = ner_labels
        self.topic_labels = topic_labels
        self.intent_labels = intent_labels
        self.tokenizer = tokenizer
        self.ner_label2id = ner_label2id
        self.topic_label2id = topic_label2id
        self.intent_label2id = intent_label2id
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        """
        Lấy 1 mẫu dữ liệu theo index
        
        Quy trình xử lý:
            1. Tokenize từng từ thành subword tokens
            2. Gán NER label cho subword đầu tiên, các subword sau gán -100
            3. Thêm special tokens: <s> (CLS) đầu câu, </s> (SEP) cuối câu
            4. Padding đến max_length
            5. Chuyển topic thành multi-hot vector
            6. Chuyển intent thành integer ID
        """
        words = self.texts[idx]              # List các từ: ["Em", "muốn", "thi", ...]
        ner_tags = self.ner_labels[idx]      # List NER labels: ["O", "O", "O", "B-ORG", ...]
        topic_str = self.topic_labels[idx]   # String: "ADMISSION|MAJOR"
        intent_str = self.intent_labels[idx] # String: "ask_info"
        
        # ========== Tokenize và align NER labels ==========
        tokens = []        # Lưu subword tokens
        ner_label_ids = [] # Lưu NER label IDs tương ứng
        
        # Thêm token đặc biệt <s> (CLS) ở đầu
        tokens.append(self.tokenizer.cls_token)
        ner_label_ids.append(-100)  # -100 = ignore token này khi tính loss
        
        # Tokenize từng từ
        for word, label in zip(words, ner_tags):
            # PhoBERT có thể tách 1 từ thành nhiều subwords
            # VD: "Bách_Khoa" -> ["Bá", "##ch_", "##Khoa"]
            word_tokens = self.tokenizer.tokenize(word)
            
            # Chỉ thêm nếu còn chỗ (giới hạn max_length)
            if len(tokens) + len(word_tokens) < self.max_length - 1:
                tokens.extend(word_tokens)
                
                # Subword đầu tiên nhận label thật, các subword sau nhận -100
                ner_label_ids.append(self.ner_label2id[label])
                if len(word_tokens) > 1:
                    ner_label_ids.extend([-100] * (len(word_tokens) - 1))
        
        # Thêm token đặc biệt </s> (SEP) ở cuối
        tokens.append(self.tokenizer.sep_token)
        ner_label_ids.append(-100)
        
        # ========== Chuyển tokens thành IDs ==========
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_ids)  # 1 = token thật, 0 = padding
        
        # ========== Padding để đủ max_length ==========
        padding_length = self.max_length - len(input_ids)
        input_ids += [self.tokenizer.pad_token_id] * padding_length
        attention_mask += [0] * padding_length
        ner_label_ids += [-100] * padding_length
        
        # ========== Xử lý Topic labels (multi-label) ==========
        # Tạo binary vector: [0,0,1,0,1,...] (1 = thuộc topic đó, 0 = không)
        topic_vector = [0] * len(self.topic_label2id)
        topics = topic_str.split('|')  # Tách "ADMISSION|MAJOR" -> ["ADMISSION", "MAJOR"]
        for topic in topics:
            if topic in self.topic_label2id:
                topic_vector[self.topic_label2id[topic]] = 1
        
        # ========== Xử lý Intent label (single-label) ==========
        # Chuyển thành integer ID
        intent_id = self.intent_label2id[intent_str]
        
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'ner_labels': torch.tensor(ner_label_ids, dtype=torch.long),
            'topic_labels': torch.tensor(topic_vector, dtype=torch.long),
            'intent_labels': torch.tensor(intent_id, dtype=torch.long)
        }

# Tạo datasets (bao gồm cả test dataset)
train_dataset = MultiTaskDataset(
    train_texts, train_ner_labels, train_topic_labels, train_intent_labels,
    tokenizer, ner_label2id, topic_label2id, intent_label2id
)
val_dataset = MultiTaskDataset(
    val_texts, val_ner_labels, val_topic_labels, val_intent_labels,
    tokenizer, ner_label2id, topic_label2id, intent_label2id
)
test_dataset = MultiTaskDataset(
    test_texts, test_ner_labels, test_topic_labels, test_intent_labels,
    tokenizer, ner_label2id, topic_label2id, intent_label2id
)

print("="*80)
print(f"Train dataset:      {len(train_dataset):5d} samples")
print(f"Validation dataset: {len(val_dataset):5d} samples")
print(f"Test dataset:       {len(test_dataset):5d} samples (KHÔNG dùng trong training)")
print("="*80)

# Kiểm tra một mẫu
sample = train_dataset[0]
print(f"\nSample shapes:")
print(f"  Input IDs: {sample['input_ids'].shape}")
print(f"  NER labels: {sample['ner_labels'].shape}")
print(f"  Topic labels: {sample['topic_labels'].shape} (multi-hot vector)")
print(f"  Intent label: {sample['intent_labels'].shape} (single value)")

Train dataset:       1630 samples
Validation dataset:   349 samples
Test dataset:         350 samples (KHÔNG dùng trong training)

Sample shapes:
  Input IDs: torch.Size([256])
  NER labels: torch.Size([256])
  Topic labels: torch.Size([10]) (multi-hot vector)
  Intent label: torch.Size([]) (single value)


## 7. Định nghĩa Metrics

In [10]:
def compute_multi_task_metrics(pred):
    """
    Tính metrics cho cả 3 tasks: NER, Topic, Intent
    """
    predictions = pred.predictions
    labels = pred.label_ids
    
    # predictions và labels là tuple: (ner, topic, intent)
    # Convert to numpy if they are tensors
    ner_preds, topic_preds, intent_preds = predictions
    ner_labels, topic_labels, intent_labels = labels
    
    # Convert tensors to numpy if needed
    if torch.is_tensor(ner_preds):
        ner_preds = ner_preds.cpu().numpy()
    if torch.is_tensor(topic_preds):
        topic_preds = topic_preds.cpu().numpy()
    if torch.is_tensor(intent_preds):
        intent_preds = intent_preds.cpu().numpy()
    if torch.is_tensor(ner_labels):
        ner_labels = ner_labels.cpu().numpy()
    if torch.is_tensor(topic_labels):
        topic_labels = topic_labels.cpu().numpy()
    if torch.is_tensor(intent_labels):
        intent_labels = intent_labels.cpu().numpy()
    
    results = {}
    
    # ========== NER Metrics ==========
    ner_predictions = np.argmax(ner_preds, axis=2)
    
    # Loại bỏ các token đặc biệt (label = -100)
    true_ner_labels = []
    true_ner_predictions = []
    
    for prediction, label in zip(ner_predictions, ner_labels):
        true_label = []
        true_pred = []
        for p, l in zip(prediction, label):
            if l != -100:
                true_label.append(ner_id2label[l])
                true_pred.append(ner_id2label[p])
        true_ner_labels.append(true_label)
        true_ner_predictions.append(true_pred)
    
    # Tính NER metrics
    ner_precision = ner_precision_score(true_ner_labels, true_ner_predictions)
    ner_recall = ner_recall_score(true_ner_labels, true_ner_predictions)
    ner_f1 = ner_f1_score(true_ner_labels, true_ner_predictions)
    
    # Tính NER accuracy (token-level)
    total_tokens = sum(len(seq) for seq in true_ner_labels)
    correct_tokens = sum(1 for true_seq, pred_seq in zip(true_ner_labels, true_ner_predictions) 
                        for t, p in zip(true_seq, pred_seq) if t == p)
    ner_accuracy = correct_tokens / total_tokens if total_tokens > 0 else 0
    
    results.update({
        'ner_precision': ner_precision,
        'ner_recall': ner_recall,
        'ner_f1': ner_f1,
        'ner_accuracy': ner_accuracy
    })
    
    # ========== Topic Metrics (Multi-label) ==========
    # Chuyển logits thành binary predictions (threshold = 0.5)
    topic_predictions = (torch.sigmoid(torch.tensor(topic_preds)) > 0.5).numpy().astype(int)
    
    # Tính metrics cho từng topic label
    topic_precision, topic_recall, topic_f1, _ = precision_recall_fscore_support(
        topic_labels, topic_predictions, average='samples', zero_division=0
    )
    
    # Tính accuracy (exact match)
    topic_accuracy = accuracy_score(topic_labels, topic_predictions)
    
    results.update({
        'topic_precision': topic_precision,
        'topic_recall': topic_recall,
        'topic_f1': topic_f1,
        'topic_accuracy': topic_accuracy
    })
    
    # ========== Intent Metrics (Single-label) ==========
    intent_predictions = np.argmax(intent_preds, axis=1)
    
    intent_precision, intent_recall, intent_f1, _ = precision_recall_fscore_support(
        intent_labels, intent_predictions, average='weighted', zero_division=0
    )
    
    intent_accuracy = accuracy_score(intent_labels, intent_predictions)
    
    results.update({
        'intent_precision': intent_precision,
        'intent_recall': intent_recall,
        'intent_f1': intent_f1,
        'intent_accuracy': intent_accuracy
    })
    
    # ========== Overall Metrics ==========
    # Trung bình F1 của cả 3 tasks
    overall_f1 = (ner_f1 + topic_f1 + intent_f1) / 3
    results['overall_f1'] = overall_f1
    
    return results

print("Multi-Task Metrics function defined!")

Multi-Task Metrics function defined!


## 11. Feature Fusion Model

### 11.1. Định nghĩa Class

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaModel


class MultiTaskPhoBERT_WithFusion(nn.Module):
    """
    Multi-Task PhoBERT với Feature Fusion (Simplified)
    - Giữ 2 dropout (dropout & dropout_heavy)
    - Giữ ner_attention
    - Dùng BCEWithLogitsLoss cho Topic
    - THÊM lại aux_topic_classifier: dự đoán topic trực tiếp từ NER features
    """
    def __init__(self, phobert_path, num_ner_labels, num_topic_labels, num_intent_labels, dropout=0.2):
        super(MultiTaskPhoBERT_WithFusion, self).__init__()
        
        # Load PhoBERT
        self.phobert = RobertaModel.from_pretrained(phobert_path)
        self.phobert.config.hidden_dropout_prob = 0.25
        self.phobert.config.attention_probs_dropout_prob = 0.25
        self.hidden_size = self.phobert.config.hidden_size
        self.num_ner_labels = num_ner_labels
        
        # Dropouts: GIỮ 2 loại
        self.dropout = nn.Dropout(dropout)
        self.dropout_heavy = nn.Dropout(dropout * 1.5)
        
        # NER head
        self.ner_hidden = nn.Linear(self.hidden_size, self.hidden_size // 2)
        self.ner_norm = nn.LayerNorm(self.hidden_size // 2)
        self.ner_classifier = nn.Linear(self.hidden_size // 2, num_ner_labels)
        
        # Intent head
        self.intent_hidden = nn.Linear(self.hidden_size, self.hidden_size // 2)
        self.intent_norm = nn.LayerNorm(self.hidden_size // 2)
        self.intent_classifier = nn.Linear(self.hidden_size // 2, num_intent_labels)
        
        # Topic Fusion Head (CLS + NER features)
        fusion_input_size = self.hidden_size + num_ner_labels
        self.topic_input_proj = nn.Linear(fusion_input_size, self.hidden_size)
        
        self.topic_layer1 = nn.Sequential(
            nn.Linear(self.hidden_size, self.hidden_size),
            nn.LayerNorm(self.hidden_size),
            nn.GELU(),
            nn.Dropout(dropout * 0.5)
        )
        
        # self.topic_layer2 = nn.Sequential(
        #     nn.Linear(self.hidden_size, self.hidden_size),
        #     nn.LayerNorm(self.hidden_size),
        #     nn.GELU(),
        #     nn.Dropout(dropout * 0.5)
        # )
        
        self.topic_classifier = nn.Linear(self.hidden_size, num_topic_labels)
        
        # GIỮ ner_attention
        self.ner_attention = nn.Sequential(
            nn.Linear(num_ner_labels, num_ner_labels // 2),
            nn.Tanh(),
            nn.Linear(num_ner_labels // 2, num_ner_labels),
            nn.Softmax(dim=-1)
        )
        
        # Cross-Attention: CLS attend to sequence context
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=self.hidden_size,
            num_heads=8,
            dropout=dropout,
            batch_first=True
        )
        self.cross_attn_norm = nn.LayerNorm(self.hidden_size)
        
        # AUX HEAD: dự đoán topic trực tiếp từ NER features
        self.aux_topic_classifier = nn.Sequential(
            nn.Linear(num_ner_labels, num_ner_labels // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(num_ner_labels // 2, num_topic_labels)
        )
    
    def extract_ner_features(self, ner_logits, attention_mask):
        """
        Trích xuất NER features với attention mechanism
        MAX + AVG pooling + ner_attention
        """
        # [batch, seq_len, num_ner_labels]
        ner_probs = F.softmax(ner_logits, dim=-1)
        
        # Mask padding
        attention_mask_expanded = attention_mask.unsqueeze(-1).expand_as(ner_probs)
        ner_probs_masked = ner_probs * attention_mask_expanded
        
        # MAX pooling → entity mạnh nhất cho từng label
        # [batch, num_ner_labels]
        max_features, _ = ner_probs_masked.max(dim=1)
        
        # AVG pooling → phân bố tổng thể label trong câu
        seq_lengths = attention_mask.sum(dim=1, keepdim=True).clamp(min=1)
        avg_features = ner_probs_masked.sum(dim=1) / seq_lengths
        
        # Kết hợp max + avg
        ner_features = 0.5 * max_features + 0.5 * avg_features
        
        # ner_attention: học weight cho từng entity type
        attention_weights = self.ner_attention(ner_features)
        ner_features_weighted = ner_features * attention_weights
        
        return ner_features_weighted
    
    def forward(self, input_ids, attention_mask=None, ner_labels=None, topic_labels=None, intent_labels=None):
        # PhoBERT outputs
        outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state             # [batch, seq_len, hidden]
        cls_output = sequence_output[:, 0, :]                   # [batch, hidden]
        
        # Cross-Attention: CLS attend to full sequence
        cls_expanded = cls_output.unsqueeze(1)  # [batch, 1, hidden]
        attn_output, _ = self.cross_attention(
            query=cls_expanded,
            key=sequence_output,
            value=sequence_output,
            key_padding_mask=(attention_mask == 0) if attention_mask is not None else None
        )
        cls_output = self.cross_attn_norm(cls_output + attn_output.squeeze(1))
        
        # GIỮ 2 dropout
        cls_output_dropped = self.dropout_heavy(cls_output)
        
        # NER predictions
        ner_hidden = self.ner_hidden(sequence_output)
        ner_hidden = self.ner_norm(ner_hidden)
        ner_hidden = F.gelu(ner_hidden)
        ner_hidden = self.dropout(ner_hidden)
        ner_logits = self.ner_classifier(ner_hidden)            # [batch, seq_len, num_ner_labels]
        
        # NER features + attention
        ner_features = self.extract_ner_features(ner_logits, attention_mask)
        
        # Fusion CLS + NER features
        topic_input = torch.cat([cls_output_dropped, ner_features], dim=-1)
        
        # Topic fusion với 2 residual block
        topic_hidden = self.topic_input_proj(topic_input)
        topic_hidden = topic_hidden + self.topic_layer1(topic_hidden)
        # topic_hidden = topic_hidden + self.topic_layer2(topic_hidden)
        
        topic_logits = self.topic_classifier(topic_hidden)      # [batch, num_topic_labels]
        
        # Intent predictions
        intent_hidden = self.intent_hidden(cls_output_dropped)
        intent_hidden = self.intent_norm(intent_hidden)
        intent_hidden = F.gelu(intent_hidden)
        intent_hidden = self.dropout(intent_hidden)
        intent_logits = self.intent_classifier(intent_hidden)
        
        total_loss = None
        if ner_labels is not None and topic_labels is not None and intent_labels is not None:
            # --- NER loss ---
            loss_fct_ner = nn.CrossEntropyLoss(
                ignore_index=-100,
                label_smoothing=0.1
            )
            active_loss = attention_mask.view(-1) == 1
            active_logits = ner_logits.view(-1, self.num_ner_labels)
            active_labels = torch.where(
                active_loss,
                ner_labels.view(-1),
                torch.tensor(loss_fct_ner.ignore_index).type_as(ner_labels)
            )
            ner_loss = loss_fct_ner(active_logits, active_labels)
            
            # --- Topic loss: DÙNG BCEWithLogitsLoss CHUẨN ---
            topic_targets = topic_labels.float()
            loss_fct_topic = nn.BCEWithLogitsLoss()
            topic_loss = loss_fct_topic(topic_logits, topic_targets)
            
            # --- AUX topic loss: dự đoán topic trực tiếp từ NER features ---
            aux_topic_logits = self.aux_topic_classifier(ner_features)   # [batch, num_topic_labels]
            aux_topic_loss = F.binary_cross_entropy_with_logits(aux_topic_logits, topic_targets)
            
            # --- Intent loss ---
            loss_fct_intent = nn.CrossEntropyLoss(label_smoothing=0.1)
            intent_loss = loss_fct_intent(intent_logits, intent_labels)
            
            # Có thể chỉnh lại các weight này theo kết quả thực tế
            total_loss = (
                1.2 * ner_loss +      # tăng nhẹ weight cho NER
                4.5 * topic_loss +    # giảm nhẹ so với 4.5 cho bớt "đè" NER
                0.6 * intent_loss +
                1.0 * aux_topic_loss  # aux vừa phải, không quá gắt
            )
        
        return {
            'loss': total_loss,
            'ner_logits': ner_logits,
            'topic_logits': topic_logits,
            'intent_logits': intent_logits
        }


### 11.2. Khởi tạo Model

In [12]:
# Khởi tạo model fusion với OPTIMAL REGULARIZATION
model_fusion = MultiTaskPhoBERT_WithFusion(
    phobert_path=phobert_path,
    num_ner_labels=len(ner_label2id),
    num_topic_labels=len(topic_label2id),
    num_intent_labels=len(intent_label2id),
    dropout=0.3  # 0.35 -> 0.3: Tìm sweet spot giữa regularization và performance
).to(device)

print(f"Model initialized: {sum(p.numel() for p in model_fusion.parameters()):,} parameters")
print("Regularization: OPTIMAL (dropout=0.3)")

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model initialized: 139,769,441 parameters
Regularization: OPTIMAL (dropout=0.3)


### 11.3. Load Checkpoint

In [13]:
# Kiểm tra và load best checkpoint từ lần training trước (nếu có)
import glob
import os

checkpoint_dir = "/kaggle/working/phobert_multitask_fusion"
best_checkpoint = None

if os.path.exists(checkpoint_dir):
    # Tìm tất cả checkpoints
    checkpoints = glob.glob(f"{checkpoint_dir}/checkpoint-*")
    
    if checkpoints:
        # Sắp xếp theo số step
        checkpoints.sort(key=lambda x: int(x.split('-')[-1]))
        
        # Tìm checkpoint có eval_overall_f1 cao nhất
        best_f1 = 0
        for ckpt in checkpoints:
            trainer_state_file = os.path.join(ckpt, "trainer_state.json")
            if os.path.exists(trainer_state_file):
                import json
                with open(trainer_state_file, 'r') as f:
                    state = json.load(f)
                    # Lấy best metric từ log_history
                    for log in state.get('log_history', []):
                        if 'eval_overall_f1' in log:
                            f1 = log['eval_overall_f1']
                            if f1 > best_f1:
                                best_f1 = f1
                                best_checkpoint = ckpt
        
        if best_checkpoint:
            print("=" * 70)
            print("TÌM THẤY CHECKPOINT TỐT NHẤT TỪ LẦN TRAINING TRƯỚC!")
            print("=" * 70)
            print(f"Checkpoint: {best_checkpoint}")
            print(f"Best Overall F1: {best_f1:.4f}")
            
            # Load checkpoint vào model
            try:
                model_fusion.load_state_dict(torch.load(f"{best_checkpoint}/pytorch_model.bin", map_location=device))
                print("\nLoad checkpoint thành công!")
                print("Model sẽ tiếp tục training từ checkpoint này")
                print("=" * 70)
            except Exception as e:
                print(f"\nKhông thể load checkpoint: {e}")
                print("Sẽ training từ đầu với PhoBERT pretrained")
        else:
            print("Không tìm thấy checkpoint với eval_overall_f1")
            print("Sẽ training từ đầu với PhoBERT pretrained")
    else:
        print("Không có checkpoint nào trong thư mục")
        print("Sẽ training từ đầu với PhoBERT pretrained")
else:
    print("Thư mục checkpoint không tồn tại")
    print("Sẽ training từ đầu với PhoBERT pretrained")

Thư mục checkpoint không tồn tại
Sẽ training từ đầu với PhoBERT pretrained


### 11.4. Cấu hình Training

In [14]:
# Training arguments với Early Stopping và Performance Optimization
from transformers import EarlyStoppingCallback

training_args_fusion = TrainingArguments(
    output_dir="./phobert_multitask_fusion",
    num_train_epochs=100,
    
    # BATCH SIZE OPTIMIZATION: Giảm để học chậm, tập trung hơn
    per_device_train_batch_size=24,  # 32 -> 24: Smaller batch = noisier gradient = better generalization
    per_device_eval_batch_size=48,   # 64 -> 48: Match train batch
    gradient_accumulation_steps=2,    # Effective batch = 24 * 2 = 48
    
    # LEARNING RATE OPTIMIZATION - CÂN BẰNG
    learning_rate=1.5e-5,            # 1e-5 -> 1.5e-5: Tăng để học nhanh hơn
    warmup_ratio=0.15,               # 0.2 -> 0.15: Giảm warmup
    lr_scheduler_type="cosine",      # Cosine decay tốt hơn linear
    weight_decay=0.05,               # 0.06 -> 0.05: Giảm để tăng flexibility
    
    # MIXED PRECISION TRAINING: Tăng tốc 2-3x
    fp16=True,                       # Bật mixed precision (float16)
    fp16_opt_level="O1",            # Optimization level 1 (safe)
    
    # EVALUATION & CHECKPOINTING - FREQUENT
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=150,                  # 200 -> 150: Đánh giá thường xuyên hơn
    save_strategy="steps",
    save_steps=150,                  # 200 -> 150: Save thường xuyên hơn
    save_total_limit=5,              # 3 -> 5: Giữ nhiều checkpoint hơn
    load_best_model_at_end=True,
    metric_for_best_model="eval_overall_f1",
    greater_is_better=True,
    
    # PERFORMANCE OPTIMIZATION
    dataloader_num_workers=4,        # Tăng từ 2 -> 4 workers
    dataloader_pin_memory=True,      # Pin memory cho GPU transfer nhanh hơn
    group_by_length=True,            # Group samples theo độ dài -> ít padding
    gradient_checkpointing=False,    # Tắt để tăng tốc (dùng khi không bị OOM)
    
    report_to="none",
    remove_unused_columns=False
)

print("=" * 70)
print("TRAINING CONFIG - BALANCED FOR 85%+ F1 (v2)")
print("=" * 70)
print(f"Regularization: Dropout=0.35 | Weight_Decay=0.06 (BALANCED)")
print(f"Label Smoothing: NER=0.15 | Intent=0.15 (balanced)")
print(f"Learning Rate: 1e-5 (cosine, 20% warmup) - VERY CONSERVATIVE")
print(f"Batch Size: 32 x 2 accumulation = 64 effective")
print(f"Mixed Precision: FP16 enabled (2-3x faster)")
print(f"Data Loading: 4 workers + pin memory + group by length")
print(f"Evaluation: Every 150 steps (more frequent)")
print(f"Early Stop: patience=7 | Overfit: gap>1.35x patience=5")
print(f"Target: Overall F1 >= 85% | Delay overfit to 2000+ steps")
print("=" * 70)

TRAINING CONFIG - BALANCED FOR 85%+ F1 (v2)
Regularization: Dropout=0.35 | Weight_Decay=0.06 (BALANCED)
Label Smoothing: NER=0.15 | Intent=0.15 (balanced)
Learning Rate: 1e-5 (cosine, 20% warmup) - VERY CONSERVATIVE
Batch Size: 32 x 2 accumulation = 64 effective
Mixed Precision: FP16 enabled (2-3x faster)
Data Loading: 4 workers + pin memory + group by length
Evaluation: Every 150 steps (more frequent)
Early Stop: patience=7 | Overfit: gap>1.35x patience=5
Target: Overall F1 >= 85% | Delay overfit to 2000+ steps


### 11.5. Khởi tạo Trainer

In [15]:
class MultiTaskTrainer(Trainer):
    """
    Custom Trainer cho Multi-Task Learning
    """
    
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Tính tổng loss từ 3 tasks
        """
        # Forward pass
        outputs = model(**inputs)
        
        # Loss đã được tính trong model.forward()
        loss = outputs['loss']
        
        return (loss, outputs) if return_outputs else loss
    
    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        """
        Custom prediction step để xử lý multi-task outputs
        """
        model.eval()
        
        inputs = self._prepare_inputs(inputs)
        
        with torch.no_grad():
            outputs = model(**inputs)
            loss = outputs['loss']
            
            ner_logits = outputs['ner_logits']
            topic_logits = outputs['topic_logits']
            intent_logits = outputs['intent_logits']
        
        if prediction_loss_only:
            return (loss, None, None)
        
        # Keep as tensors, don't convert to numpy yet
        # Stack logits as tuple of tensors
        logits = (
            ner_logits.detach(),
            topic_logits.detach(),
            intent_logits.detach()
        )
        
        # Stack labels as tuple of tensors
        labels = (
            inputs['ner_labels'].detach(),
            inputs['topic_labels'].detach(),
            inputs['intent_labels'].detach()
        )
        
        return (loss, logits, labels)

print("Multi-Task Trainer class defined!")

Multi-Task Trainer class defined!


In [16]:
# Data collator cho multi-task
def multi_task_data_collator(features):
    """Collate batch data cho multi-task learning"""
    batch = {}
    batch['input_ids'] = torch.stack([f['input_ids'] for f in features])
    batch['attention_mask'] = torch.stack([f['attention_mask'] for f in features])
    batch['ner_labels'] = torch.stack([f['ner_labels'] for f in features])
    batch['topic_labels'] = torch.stack([f['topic_labels'] for f in features])
    batch['intent_labels'] = torch.stack([f['intent_labels'] for f in features])
    return batch

# 1. Early Stopping Callback: Dừng khi không cải thiện
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=5,
    early_stopping_threshold=0.0001
)

# 2. Overfit Detection Callback: Dừng khi validation loss tăng quá cao so với training loss
class OverfitDetectionCallback(TrainerCallback):
    def __init__(self, max_loss_gap=1.5, patience=3):
        """
        max_loss_gap: Ngưỡng gap giữa val_loss/train_loss
        patience: Số lần liên tiếp gap vượt ngưỡng trước khi dừng
        """
        self.max_loss_gap = max_loss_gap
        self.patience = patience
        self.overfit_count = 0
        self.train_loss_history = []
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and 'loss' in logs:
            self.train_loss_history.append({'step': state.global_step, 'train_loss': logs['loss']})
    
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if not metrics or not self.train_loss_history:
            return
        
        val_loss = metrics.get('eval_loss', 0)
        train_loss = self.train_loss_history[-1]['train_loss']
        loss_gap = val_loss / train_loss if train_loss > 0 else 1.0
        
        if loss_gap > self.max_loss_gap:
            self.overfit_count += 1
            print(f"\nOVERFIT WARNING - Step {state.global_step}:")
            print(f"  Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Gap: {loss_gap:.2f}x")
            print(f"  Overfit count: {self.overfit_count}/{self.patience}")
            
            if self.overfit_count >= self.patience:
                print(f"\nSTOPPING: Overfit detected {self.patience} times consecutively")
                print(f"Best model will be loaded automatically")
                control.should_training_stop = True
        else:
            if self.overfit_count > 0:
                print(f"Gap normalized: {loss_gap:.2f}x - Reset overfit count")
            self.overfit_count = 0

overfit_detector = OverfitDetectionCallback(
    max_loss_gap=1.5,
    patience=3
)

# Khởi tạo trainer
trainer_fusion = MultiTaskTrainer(
    model=model_fusion,
    args=training_args_fusion,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=multi_task_data_collator,
    compute_metrics=compute_multi_task_metrics,
    callbacks=[early_stopping, overfit_detector]
)

print("Trainer initialized successfully")
print("Callback 1: Early Stopping - patience=5 evaluations")
print("Callback 2: Overfit Detector - max_loss_gap=1.5x, patience=3 evaluations")

Trainer initialized successfully
Callback 1: Early Stopping - patience=5 evaluations
Callback 2: Overfit Detector - max_loss_gap=1.5x, patience=3 evaluations


### 11.6. Training

In [17]:
# Training - Model học quy luật NER -> Topic mỗi epoch
print("="*80)
print("Training Feature Fusion Model")
print("Model will learn NER -> Topic relationships during each epoch")
print("="*80)

start_time = time.time()
train_result = trainer_fusion.train()
training_time = (time.time() - start_time) / 60

print(f"\nTraining completed in {training_time:.2f} minutes")
print(f"Final loss: {train_result.training_loss:.4f}")

Training Feature Fusion Model
Model will learn NER -> Topic relationships during each epoch


Step,Training Loss,Validation Loss,Ner Precision,Ner Recall,Ner F1,Ner Accuracy,Topic Precision,Topic Recall,Topic F1,Topic Accuracy,Intent Precision,Intent Recall,Intent F1,Intent Accuracy,Overall F1
150,5.063700,4.338528,0.187500,0.001315,0.002611,0.753778,0.769341,0.561127,0.617670,0.315186,0.629033,0.759312,0.686512,0.759312,0.435597
300,3.543100,3.250881,0.525358,0.417616,0.465332,0.863998,0.857211,0.781519,0.792455,0.558739,0.779351,0.836676,0.806530,0.836676,0.688106
450,2.919400,2.947485,0.646913,0.684049,0.664963,0.915832,0.894460,0.831662,0.844017,0.630372,0.802877,0.845272,0.820038,0.845272,0.776339
600,2.584300,2.862535,0.695257,0.770815,0.731089,0.932026,0.871681,0.861987,0.849129,0.647564,0.840447,0.848138,0.833694,0.848138,0.804637
750,2.273700,2.846398,0.732544,0.813760,0.771019,0.939446,0.881328,0.858644,0.851085,0.650430,0.865789,0.876791,0.867896,0.876791,0.830000
900,2.087200,2.920639,0.744038,0.820333,0.780325,0.942913,0.880850,0.876313,0.858610,0.636103,0.885485,0.871060,0.874243,0.871060,0.837726
1050,1.979900,2.938799,0.752871,0.833041,0.790930,0.943888,0.891834,0.858883,0.856774,0.653295,0.884726,0.876791,0.879651,0.876791,0.842452
1200,1.902800,3.024415,0.758471,0.843558,0.798755,0.946650,0.876552,0.868434,0.854400,0.653295,0.876844,0.865330,0.866622,0.865330,0.839926
1350,1.850200,3.055462,0.761792,0.849255,0.803150,0.947300,0.881328,0.874164,0.859353,0.661891,0.896121,0.879656,0.884502,0.879656,0.849002
1500,1.820800,3.081428,0.774631,0.851008,0.811025,0.947950,0.894222,0.874642,0.865862,0.659026,0.889248,0.873926,0.879113,0.873926,0.852000



OVERFIT WARNING - Step 1200:
  Train Loss: 1.9028 | Val Loss: 3.0244 | Gap: 1.59x
  Overfit count: 1/3

OVERFIT WARNING - Step 1350:
  Train Loss: 1.8502 | Val Loss: 3.0555 | Gap: 1.65x
  Overfit count: 2/3

OVERFIT WARNING - Step 1500:
  Train Loss: 1.8208 | Val Loss: 3.0814 | Gap: 1.69x
  Overfit count: 3/3

STOPPING: Overfit detected 3 times consecutively
Best model will be loaded automatically

Training completed in 29.13 minutes
Final loss: 2.8981


### 11.7. Đánh giá Test Set

In [18]:
# Evaluate on test set
test_results_fusion = trainer_fusion.evaluate(test_dataset)

print("="*80)
print("Test Results - Feature Fusion Model")
print("="*80)
print(f"Overall F1: {test_results_fusion['eval_overall_f1']:.4f}")
print(f"NER F1: {test_results_fusion['eval_ner_f1']:.4f}")
print(f"Topic F1: {test_results_fusion['eval_topic_f1']:.4f}")
print(f"Topic Recall: {test_results_fusion['eval_topic_recall']:.4f}")
print(f"Intent F1: {test_results_fusion['eval_intent_f1']:.4f}")
print("="*80)


OVERFIT WARNING - Step 1500:
  Train Loss: 1.8208 | Val Loss: 3.2415 | Gap: 1.78x
  Overfit count: 4/3

STOPPING: Overfit detected 3 times consecutively
Best model will be loaded automatically
Test Results - Feature Fusion Model
Overall F1: 0.8335
NER F1: 0.8232
Topic F1: 0.8598
Topic Recall: 0.8691
Intent F1: 0.8176


### 11.8. Lưu Model

In [19]:
# Hàm tìm best checkpoint
def find_best_checkpoint(output_dir):
    """Tìm checkpoint tốt nhất dựa trên tên folder"""
    import glob
    checkpoints = glob.glob(f"{output_dir}/checkpoint-*")
    if not checkpoints:
        return None
    # Sắp xếp theo số step (checkpoint-xxx)
    checkpoints.sort(key=lambda x: int(x.split('-')[-1]))
    return checkpoints[-1] if checkpoints else None

# Save model
fusion_final_path = "./phobert_multitask_fusion_final"
os.makedirs(fusion_final_path, exist_ok=True)

torch.save(model_fusion.state_dict(), f"{fusion_final_path}/pytorch_model.bin")
tokenizer.save_pretrained(fusion_final_path)

# Save label mappings
with open(f"{fusion_final_path}/label_mappings.json", 'w', encoding='utf-8') as f:
    json.dump({
        'ner_label2id': ner_label2id,
        'ner_id2label': ner_id2label,
        'topic_label2id': topic_label2id,
        'topic_id2label': topic_id2label,
        'intent_label2id': intent_label2id,
        'intent_id2label': intent_id2label
    }, f, ensure_ascii=False, indent=2)

# Save config
model_config_fusion = {
    'model_type': 'MultiTaskPhoBERT_WithFusion',
    'phobert_base': phobert_path,
    'num_ner_labels': len(ner_label2id),
    'num_topic_labels': len(topic_label2id),
    'num_intent_labels': len(intent_label2id),
    'hidden_size': model_fusion.phobert.config.hidden_size,
    'dropout': 0.1,
    'test_results': test_results_fusion
}

with open(f"{fusion_final_path}/model_config.json", 'w', encoding='utf-8') as f:
    json.dump(model_config_fusion, f, ensure_ascii=False, indent=2)

print(f"Model saved to: {fusion_final_path}")

Model saved to: ./phobert_multitask_fusion_final


## 12. Download Model

In [20]:
# ========== TẢI MODEL VỀ MÁY CÁ NHÂN ==========

if IS_KAGGLE:
    import shutil
    from pathlib import Path
    
    print("="*80)
    print("CHUẨN BỊ DOWNLOAD MODEL VỀ MÁY CÁ NHÂN")
    print("="*80)
    
    # Đường dẫn model đã train (từ checkpoint tốt nhất hoặc final)
    source_paths = [
        "/kaggle/working/phobert_multitask_fusion_final",  # Model final
        "/kaggle/working/phobert_multitask_fusion_model",  # Thư mục checkpoints
    ]
    
    # Tạo file zip để dễ download
    zip_output = "/kaggle/working/phobert_model_download"
    
    print("\nĐang nén model...")
    
    # Nén model final
    if os.path.exists(source_paths[0]):
        shutil.make_archive(
            "/kaggle/working/phobert_multitask_final",
            'zip',
            source_paths[0]
        )
        print(f"Đã nén: phobert_multitask_final.zip")
        print(f"  Kích thước: {os.path.getsize('/kaggle/working/phobert_multitask_final.zip') / (1024*1024):.1f} MB")
    
    # Nén checkpoint tốt nhất
    best_checkpoint = find_best_checkpoint("/kaggle/working/phobert_multitask_fusion")
    if best_checkpoint:
        checkpoint_name = os.path.basename(best_checkpoint)
        shutil.make_archive(
            f"/kaggle/working/{checkpoint_name}",
            'zip',
            best_checkpoint
        )
        print(f"Đã nén: {checkpoint_name}.zip")
        print(f"  Kích thước: {os.path.getsize(f'/kaggle/working/{checkpoint_name}.zip') / (1024*1024):.1f} MB")
    
    
    # Liệt kê các file có thể download
    print("\nCÁC FILE TRONG /kaggle/working:")
    print("-" * 80)
    for file in os.listdir("/kaggle/working"):
        if file.endswith('.zip') or os.path.isdir(f"/kaggle/working/{file}"):
            path = f"/kaggle/working/{file}"
            if os.path.isfile(path):
                size = os.path.getsize(path) / (1024*1024)
                print(f"   {file:50s} ({size:.1f} MB)")
            else:
                print(f"   {file}/")
    
    print("="*80)
    
else:
    print("Code này chỉ chạy trên Kaggle")
    print("Trên Local, model đã có sẵn tại: ./phobert_multitask_final")

CHUẨN BỊ DOWNLOAD MODEL VỀ MÁY CÁ NHÂN

Đang nén model...
Đã nén: phobert_multitask_final.zip
  Kích thước: 496.2 MB
Đã nén: checkpoint-1500.zip
  Kích thước: 1160.5 MB

CÁC FILE TRONG /kaggle/working:
--------------------------------------------------------------------------------
   phobert_multitask_fusion_final/
   phobert_multitask_final.zip                        (496.2 MB)
   phobert_multitask_fusion/
   checkpoint-1500.zip                                (1160.5 MB)
